In [3]:
import numpy as np

# 1강

In [6]:
class Variable:
	def __init__(self, data) -> None:
		self.data = data

# 2강

In [7]:
class Function:
	def __call__(self, input):
		x = input.data
		y = self.forward(x)
		output = Variable(y)
		return output

	def forward(self, x):
		raise NotImplementedError()

# 3강

In [8]:
class Square(Function):
	def forward(self, x):
		return x ** 2

class Exp(Function):
	def forward(self, x):
		return np.exp(x)


# 4강

In [10]:
def numerical_diff(f, x, eps=1e-4):
	x0 = Variable(x.data - eps)
	x1 = Variable(x.data + eps)

	y0 = f(x0)
	y1 = f(x1)
	return (y1.data - y0.data) / (2 * eps)

def f(x):
	a = Square()
	b = Exp()
	return a(b(a(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f,x)
print(dy)



3.2974426293330694


# 5강

연쇄법칙을 활용한 역전파. 그러나 순서가 앞에서 시작돼도 분명 가능하다

# 6강

In [20]:
class Variable:
	def __init__(self, data):
		self.data = data
		self.grad = None # 미분값을 저장함.

class Function:
	def __call__(self, input):
		x = input.data
		y = self.forward(x)
		output = Variable(y)
		self.input = input # 변수 보관
		return output

	def forward(self, x):
		raise NotImplementedError()

	def backward(self, gy): # 역전파 메소드
		raise NotImplementedError()


class Square(Function):
	def forward(self, x):
		return x ** 2

	def backward(self, gy):
		x = self.input.data
		gx = 2 * x * gy
		return gx
# x 제곱의 미분 값은 2x. 즉 입력값이 활용된다.

class Exp(Function):
	def forward(self, x):
		return np.exp(x)

	def backward(self, gy):
		x = self.input.data
		gx = np.exp(x) * gy
		return gx

class Triple(Function):
	def forward(self, x):
		return x ** 3
	
	def backward(self, gy):
		x = self.input.data
		gx = 3 * (x ** 2) * gy
		return gx

a = Square()
b = Exp()
c = Triple()

x = Variable(np.array(0.5))
p = a(x)
q = b(p)
y = c(q)

print('before backpropagation')
print(x.grad)

y.grad = np.array(1.0)
q.grad = c.backward(y.grad)
p.grad = b.backward(q.grad)
x.grad = a.backward(p.grad)
print('by backpropagation')
print(x.grad) #역전파로 나오는 값

#값이 잘 나오는지 확인
def fun(x):
	y = a(x)
	y = b(y)
	y = c(y)
	return y

tmp = numerical_diff(fun, x)
print('by numerical_diff')
print(tmp)


before backpropagation
None
by backpropagation
6.351000049838023
by numerical_diff
6.351000335633739


앞에서부터 미분을 진행할 수 있을까?

In [21]:
class Variable:
	def __init__(self, data):
		self.data = data
		self.grad = None # 앞에서부터의 미분값.

class Function:
	def __call__(self, input):
		x = input.data
		y = self.forward(x)
		output = Variable(y)
		self.input = input # 변수 보관
		return output

	def forward(self, x):
		raise NotImplementedError()

	def backward(self, gy): #엄밀하게 back이 아님. 이것도 forward
		raise NotImplementedError()


class Square(Function):
	def forward(self, x):
		return x ** 2

	def backward(self, gy): #gy도 앞에서 전달되는 값
		x = self.input.data
		gx = 2 * x * gy
		return gx

class Exp(Function):
	def forward(self, x):
		return np.exp(x)

	def backward(self, gy):
		x = self.input.data
		gx = np.exp(x) * gy
		return gx

class Triple(Function):
	def forward(self, x):
		return x ** 3
	
	def backward(self, gy):
		x = self.input.data
		gx = 3 * (x ** 2) * gy
		return gx


a = Square()
b = Exp()
c = Triple()

x = Variable(np.array(0.5))
p = a(x)
q = b(p)
y = c(q)


#앞에서부터 미분을 진행한다면? dx/dx로부터 시작을 하자.
x.grad = np.array(1.0)
p.grad = a.backward(x.grad) #backward라고 써있지만.. 앞으로 간다.
q.grad = b.backward(p.grad)
y.grad = c.backward(q.grad)
print(x.grad)
print(p.grad)
print(q.grad)
print(y.grad)
#결과적으로 grad의 의미가 바뀌었고, 저장되는 위치가 바뀌게 되었다.
#grad는 각 위치의 미분값을 나타내는 것이 아님. 독립적이지 않음.

1.0
1.0
1.2840254166877414
6.351000049838023


추가적으로 생각해볼 사안. \
만약에 분기가 일어난다면? - 11강 \
최종 grad의 위치를 똑같이 하고 싶다면?

# 7강

In [26]:
class Variable:
	def __init__(self, data):
		self.data = data
		self.grad = None
		self.creator = None # 창조자를 담을 변수

	def set_creator(self, func):
		self.creator = func #자신의 창조자를 저장함

	def backward(self):
		f = self.creator # 창조자를 활용함
		if f is not None: # 창조한 함수가 없을 때까지
			x = f.input
			x.grad = f.backward(self.grad)
			x.backward()
	# 변수의 역전파 메소드는 재귀를 돈다.

class Function:
	def __call__(self, input):
		x = input.data
		y = self.forward(x)
		output = Variable(y) # 함수로 만들어진 출력이 변수에 담김
		self.input = input # 입력변수 보관

		output.set_creator(self) # 그 변수는 자신을 만든 함수를 저장함
		self.output = output # 출력변수도 보관
		return output

	def forward(self, x):
		raise NotImplementedError()

	def backward(self, gy):
		raise NotImplementedError()


class Square(Function):
	def forward(self, x):
		return x ** 2

	def backward(self, gy):
		x = self.input.data
		gx = 2 * x * gy
		return gx

class Exp(Function):
	def forward(self, x):
		return np.exp(x)

	def backward(self, gy):
		x = self.input.data
		gx = np.exp(x) * gy
		return gx

class Triple(Function):
	def forward(self, x):
		return x ** 3
	
	def backward(self, gy):
		x = self.input.data
		gx = 3 * (x ** 2) * gy
		return gx

A = Square()
B = Exp()
C = Triple()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward() # 변수 객체의 backward.
print(x.grad)



6.351000049838023


In [28]:
#당연히 원래 방법도 가능
y.grad = np.array(1.0)
q.grad = C.backward(y.grad)
p.grad = B.backward(q.grad)
x.grad = A.backward(p.grad)
print('by backpropagation')
print(x.grad) #역전파로 나오는 값

by backpropagation
6.351000049838023


와! 복습 끝! 최종적으로 만들어진 클래스들의 모습

In [ ]:
class Variable:
	def __init__(self, data):
		self.data = data
		self.grad = None
		self.creator = None

	def set_creator(self, func):
		self.creator = func

	def backward(self):
		f = self.creator
		if f is not None:
			x = f.input
			x.grad = f.backward(self.grad)
			x.backward()

class Function:
	def __call__(self, input):
		x = input.data
		y = self.forward(x)
		output = Variable(y)
		output.set_creator(self)
		self.input = input
		self.output = output #이따 쓰인다..
		return output

	def forward(self, x):
		raise NotImplementedError()

	def backward(self, gy):
		raise NotImplementedError()

이제 우리가 만들게 될 최종 코드를 보자

# 8강

재귀를 반복문으로 바꾸기

In [34]:
class Variable:
	def __init__(self, data):
		self.data = data
		self.grad = None
		self.creator = None

	def set_creator(self, func):
		self.creator = func #자신의 창조자를 저장함

# 재귀 코드
	# def backward(self):
	# 	f = self.creator
	# 	if f:
	# 		x = f.input
	# 		x.grad = f.backward(self.grad)
	# 		x.backward() # 자기 자신을 활용하기에 계속 self.grad를 활용함.


# 반복문 코드
	def backward(self):
		funcs = [self.creator]
		while funcs:
			f = funcs.pop()
			x, y = f.input, f.output # 여기에서 output 값이 활용됨.
			x.grad = f.backward(y.grad) # 우리는 output의 미분값을 알고 싶다..

			if x.creator is not None: # 시작이 아니라면
				funcs.append(x.creator)

#어차피 들어갔다가 바로 나올 거라면, list일 필요가 없다!

# 내 코드
	# def backward(self):
	# 	f = self.creator
	# 	while f:
	# 		x, y = f.input, f.output
	# 		x.grad = f.backward(y.grad)
	# 		f = x.creator

A = Square()
B = Exp()
C = Triple()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

6.351000049838023


꼬리 재귀가 뭘까?

In [43]:
#일반 재귀
def fact1(n):
	if n == 1:
		return 1
	return n * fact1(n-1)

#꼬리 재귀
def fact2(n, total = 1):
	if n == 1:
		return total
	return fact2(n-1, total * n) # 재귀 자체가 반환되도록 만든다.
# 중간 재귀들은 아무 일도 하지 않고, 전달만 한다. 

print(fact1(5))
print('-----------')
print(fact2(5))

120
-----------
120


In [44]:
def fact1(n):
	if n == 1:
		return 1
	tmp = n * fact1(n-1)
	print(tmp)
	return tmp

def fact2(n, total = 1):
	if n == 1:
		return total
	tmp = fact2(n-1, total * n) 
	print(tmp)
	return tmp 


print(fact1(5))
print('-----------')
print(fact2(5))


2
6
24
120
120
-----------
120
120
120
120
120


# 9강

함수 객체로 쓰지 말고.. 함수처럼 쓰자..

In [45]:
def square(x):
	return Square()(x) #함수화 시켜버림
def exp(x):
	return Exp()(x)
def triple(x):
	return Triple()(x)

In [47]:
x = Variable(np.array(0.5))
y = triple(exp(square(x))) #와! 함수처럼 쓸 수 있게 됐다!
y.grad = np.array(1.0)
y.backward()
print(x.grad)

6.351000049838023


dy/dy 그만 쓰고 싶다.. 허구헌 날 1 쓰고 싶지 않다..

In [49]:
class Variable:
	def __init__(self, data):
		self.data = data
		self.grad = None
		self.creator = None

	def set_creator(self, func):
		self.creator = func

	def backward(self):
		#변수에 대해 backward를 하는 것은 마지막 밖에 없음.
		if self.grad is None: 
			self.grad = np.ones_like(self.data) # 그렇기에 맨 마지막 미분값만 1이 됨

		funcs = [self.creator]
		while funcs:
			f = funcs.pop()
			x, y = f.input, f.output
			x.grad = f.backward(y.grad)

			if x.creator is not None:
				funcs.append(x.creator)


x = Variable(np.array(0.5))
y = triple(exp(square(x)))
# 귀찮게 y.grad 안 써도 된다!
y.backward()
print(x.grad)


6.351000049838023


다른 자료형은 받지 않겠다.

In [53]:
#이 코드를 실행하기 이전에 밑 코드를 한번만 먼저 실행하세요

class Variable:
	def __init__(self, data):
		# 데이터가 None도, ndarray도 아니라면?
		if data is not None and not isinstance(data, np.ndarray):
			raise TypeError(f'{type(data)} 안돼안돼 무조건 ndarray')

		self.data = data
		self.grad = None
		self.creator = None

	def set_creator(self, func):
		self.creator = func

	def backward(self):
		if self.grad is None:
			self.grad = np.ones_like(self.data)

		funcs = [self.creator]
		while funcs:
			f = funcs.pop()
			x, y = f.input, f.output
			x.grad = f.backward(y.grad)

			if x.creator is not None:
				funcs.append(x.creator)

In [54]:
print(Variable(np.array(1.0)))
print(Variable(None))
print(Variable(1.0))

TypeError: <class 'float'> 안돼안돼 무조건 ndarray

In [57]:
#문제 발생
x = Variable(np.array(0.5))
y = triple(exp(square(x)))
y.backward()
print(x.grad)

TypeError: <class 'numpy.float64'> 안돼안돼 무조건 ndarray

In [58]:
# x ** 2는 array가 아니게 된다.
def as_array(x):
	if np.isscalar(x): return np.array(x)
	return x

In [61]:
class Function:
	def __call__(self, input):
		x = input.data
		y = self.forward(x)
		output = Variable(as_array(y)) # 중요한 건 꺾이지 않는 ndarray
		output.set_creator(self)
		self.input = input
		self.output = output
		return output

	def forward(self, x):
		raise NotImplementedError()

	def backward(self, gy):
		raise NotImplementedError()
		
class Square(Function):
	def forward(self, x):
		return x ** 2

	def backward(self, gy):
		x = self.input.data
		gx = 2 * x * gy
		return gx

class Exp(Function):
	def forward(self, x):
		return np.exp(x)

	def backward(self, gy):
		x = self.input.data
		gx = np.exp(x) * gy
		return gx

class Triple(Function):
	def forward(self, x):
		return x ** 3
	
	def backward(self, gy):
		x = self.input.data
		gx = 3 * (x ** 2) * gy
		return gx


x = Variable(np.array(0.5))
y = triple(exp(square(x)))
y.backward()
print(x.grad)

6.351000049838023


# 10강

파이썬 파일로